# HPC in Finance - Performance Overview

In [ ]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats, special
from IPython.display import Image

: 

In [ ]:
v0  = [332.1]                       # default
v1  = [212.95]                      # put-call parity added
v2  = [199.9]                       # float to double (CHECK AGAIN with std::expf etc.)                
v3  = [198.25]                      # ... pass by const. reference added (e.g. const float& S) (runtime: -2%)
v4  = [192.3]                       # switch (x / std::sqrt(2.0)) -> x * 0.707...
v5  = [193.65]                      # N_CDF is inline (NO CHANGE)
v6  = [193.05]                      # put & call are inline too (NO CHANGE)
v7  = [191.1]                       # sigma_sqrt_t = sigma * std::sqrt(t) replacement
v8  = [189.2]                       # 0.5* instead of /2.0
v9  = [151.1]                       # std::pow(sigma, 2) --> sigma * sigma
v10 = [149.15]                      # std::exp(-r*t) is global variable (changed in function calls)
v11 = [144.1]                       # exp_min_t_r, sigma_sqrt_t, d_1 are all global
v12 = [143.15]                      # 0.5*(1.0 + ...) = 0.5 + 0.5*...
v13 = [123.05]                      # from std::vector<double> --> static double array
v14 = [120.3]                       # static double array -> global (defined) double array
v15 = [119.6]                       # changed conditions (> --> !=) and order s.t. most likely case comes 1st


# Unattainable
v20 = [39.25]                       # if inline were to be constant

### Idea: Replace costly `N_CDF` function that uses `std::erff` with table-like switch case

In [2]:
quantile_table = dict()

def std_erf(x):
    '''return 0.5 * (1.0 + std::erff(x / std::sqrt(2.0)));'''
    return 0.5+0.5*special.erf(x / np.sqrt(2.0))

In [3]:
# define p grid
p_grid = np.linspace(0.0,1.0,11)
q_grid = np.array([stats.norm.ppf(p_loc, 0, 1) for p_loc in p_grid])
# replace inf 
q_grid[0]  = q_grid[1]  + (q_grid[1] - q_grid[2])
q_grid[-1] = q_grid[-2] + (q_grid[-2] - q_grid[-3])
q_grid

array([-1.7214819 , -1.28155157, -0.84162123, -0.52440051, -0.2533471 ,
        0.        ,  0.2533471 ,  0.52440051,  0.84162123,  1.28155157,
        1.7214819 ])

In [4]:
# compile C++ function body
str_txt = ""
for j, q_loc in enumerate(q_grid):
    if j == 0:
        str_txt += "if(x < {}){{\n    return 0.0;\n}}".format(str(round(q_loc, 5)))
    elif((j != len(q_grid) - 1) and (q_grid[j] < 0)):
        str_txt += "else if(x < {}){{\n    alpha = ((x + {}) / ({} + {}));\n    return alpha*{} + (1-alpha)*{};\n}}".format(str(round(q_loc, 5)), 
                                                                                                                     str(round(-1*q_grid[j], 5)), 
                                                                                                                     str(round(q_grid[j-1], 5)),
                                                                                                                     str(round(-1*q_grid[j], 5)), 
                                                                                                                     str(round(p_grid[j], 5)),
                                                                                                                     str(round(p_grid[j-1], 5)))
    elif((j != len(q_grid) - 1) and (q_grid[j] >= 0)):
        str_txt += "else if(x < {}){{\n    alpha = ((x - {}) / ({} - {}));\n    return alpha*{} + (1-alpha)*{};\n}}".format(str(round(q_loc, 5)), 
                                                                                                                     str(round(q_grid[j], 5)), 
                                                                                                                     str(round(q_grid[j-1], 5)),
                                                                                                                     str(round(q_grid[j], 5)), 
                                                                                                                     str(round(p_grid[j], 5)),
                                                                                                                     str(round(p_grid[j-1], 5)))
    else:
        str_txt += "\nelse{\n    return 1.0;\n}"
        break
# print text
print(str_txt)

if(x < -1.72148){
    return 0.0;
}else if(x < -1.28155){
    alpha = ((x + 1.28155) / (-1.72148 + 1.28155));
    return alpha*0.1 + (1-alpha)*0.0;
}else if(x < -0.84162){
    alpha = ((x + 0.84162) / (-1.28155 + 0.84162));
    return alpha*0.2 + (1-alpha)*0.1;
}else if(x < -0.5244){
    alpha = ((x + 0.5244) / (-0.84162 + 0.5244));
    return alpha*0.3 + (1-alpha)*0.2;
}else if(x < -0.25335){
    alpha = ((x + 0.25335) / (-0.5244 + 0.25335));
    return alpha*0.4 + (1-alpha)*0.3;
}else if(x < 0.0){
    alpha = ((x - 0.0) / (-0.25335 - 0.0));
    return alpha*0.5 + (1-alpha)*0.4;
}else if(x < 0.25335){
    alpha = ((x - 0.25335) / (0.0 - 0.25335));
    return alpha*0.6 + (1-alpha)*0.5;
}else if(x < 0.5244){
    alpha = ((x - 0.5244) / (0.25335 - 0.5244));
    return alpha*0.7 + (1-alpha)*0.6;
}else if(x < 0.84162){
    alpha = ((x - 0.84162) / (0.5244 - 0.84162));
    return alpha*0.8 + (1-alpha)*0.7;
}else if(x < 1.28155){
    alpha = ((x - 1.28155) / (0.84162 - 1.28155));
    return 

In [5]:
# compile Python function body
str_txt = ""
for j, q_loc in enumerate(q_grid):
    if j == 0:
        str_txt += "if(x < {}):\n    return 0.0".format(str(round(q_loc, 5)))
    elif((j != len(q_grid) - 1) and (q_grid[j] < 0)):
        str_txt += "\nelif(x < {}):\n    alpha = ((x + {}) / ({} + {}))\n    return alpha*{} + (1-alpha)*{}".format(str(round(q_loc, 5)), 
                                                                                                                     str(round(-1*q_grid[j], 5)), 
                                                                                                                     str(round(q_grid[j-1], 5)),
                                                                                                                     str(round(-1*q_grid[j], 5)), 
                                                                                                                     str(round(p_grid[j], 5)),
                                                                                                                     str(round(p_grid[j-1], 5)))
    elif((j != len(q_grid) - 1) and (q_grid[j] >= 0)):
        str_txt += "\nelif(x < {}):\n    alpha = ((x - {}) / ({} - {}))\n    return alpha*{} + (1-alpha)*{}".format(str(round(q_loc, 5)), 
                                                                                                                     str(round(q_grid[j], 5)), 
                                                                                                                     str(round(q_grid[j-1], 5)),
                                                                                                                     str(round(q_grid[j], 5)), 
                                                                                                                     str(round(p_grid[j], 5)),
                                                                                                                     str(round(p_grid[j-1], 5)))
    else:
        str_txt += "\nelse:\n    return 1.0"
        break
# print text
print(str_txt)

if(x < -1.72148):
    return 0.0
elif(x < -1.28155):
    alpha = ((x + 1.28155) / (-1.72148 + 1.28155))
    return alpha*0.1 + (1-alpha)*0.0
elif(x < -0.84162):
    alpha = ((x + 0.84162) / (-1.28155 + 0.84162))
    return alpha*0.2 + (1-alpha)*0.1
elif(x < -0.5244):
    alpha = ((x + 0.5244) / (-0.84162 + 0.5244))
    return alpha*0.3 + (1-alpha)*0.2
elif(x < -0.25335):
    alpha = ((x + 0.25335) / (-0.5244 + 0.25335))
    return alpha*0.4 + (1-alpha)*0.3
elif(x < 0.0):
    alpha = ((x - 0.0) / (-0.25335 - 0.0))
    return alpha*0.5 + (1-alpha)*0.4
elif(x < 0.25335):
    alpha = ((x - 0.25335) / (0.0 - 0.25335))
    return alpha*0.6 + (1-alpha)*0.5
elif(x < 0.5244):
    alpha = ((x - 0.5244) / (0.25335 - 0.5244))
    return alpha*0.7 + (1-alpha)*0.6
elif(x < 0.84162):
    alpha = ((x - 0.84162) / (0.5244 - 0.84162))
    return alpha*0.8 + (1-alpha)*0.7
elif(x < 1.28155):
    alpha = ((x - 1.28155) / (0.84162 - 1.28155))
    return alpha*0.9 + (1-alpha)*0.8
else:
    return 1.0


In [8]:
def fake_erf(x):
    if(x < -1.72148):
        return 0.0
    elif(x < -1.28155):
        alpha = ((x + 1.28155) / (-1.72148 + 1.28155))
        return alpha*0.1 + (1-alpha)*0.0
    elif(x < -0.84162):
        alpha = ((x + 0.84162) / (-1.28155 + 0.84162))
        return alpha*0.2 + (1-alpha)*0.1
    elif(x < -0.5244):
        alpha = ((x + 0.5244) / (-0.84162 + 0.5244))
        return alpha*0.3 + (1-alpha)*0.2
    elif(x < -0.25335):
        alpha = ((x + 0.25335) / (-0.5244 + 0.25335))
        return alpha*0.4 + (1-alpha)*0.3
    elif(x < 0.0):
        alpha = ((x - 0.0) / (-0.25335 - 0.0))
        return alpha*0.5 + (1-alpha)*0.4
    elif(x < 0.25335):
        alpha = ((x - 0.25335) / (0.0 - 0.25335))
        return alpha*0.6 + (1-alpha)*0.5
    elif(x < 0.5244):
        alpha = ((x - 0.5244) / (0.25335 - 0.5244))
        return alpha*0.7 + (1-alpha)*0.6
    elif(x < 0.84162):
        alpha = ((x - 0.84162) / (0.5244 - 0.84162))
        return alpha*0.8 + (1-alpha)*0.7
    elif(x < 1.28155):
        alpha = ((x - 1.28155) / (0.84162 - 1.28155))
        return alpha*0.9 + (1-alpha)*0.8
    else:
        return 1.0


In [19]:
# Compare the two
print('Approximation fake_erf(x)=', round(fake_erf(0.2), 4), ', Groundtruth erf(x)=', round(std_erf(0.2), 4))

print('abs. error: ', round(abs(fake_erf(0.2) - std_erf(0.2)), 3))

Approximation fake_erf(x)= 0.5211 , Groundtruth erf(x)= 0.5793
abs. error:  0.058


In [21]:
# Check global performance (trivially)
f_vals = [np.abs(fake_erf(x_loc) - std_erf(x_loc)) for x_loc in np.linspace(0.0,1.0,1001)]
print('Max difference: ', round(max(f_vals), 4))

Max difference:  0.1


## Black Scholes Reference Price

In [ ]:
def N_CDF(x):
    '''CDF of Standard Normal Distribution'''
    return stats.norm.cdf(x, 0, 1)

def BS(S, K, t, r, sigma):
    assert S>=0 and K>=0 and sigma>=0 and t>=0 and r >=0, "All inputs should be nonnegative"
    assert S > 0 and K > 0, "Inputs \'S\' and \'K\' should be positive."
    d_1 = (np.log(S/K) + (r + 0.5*sigma**2)*t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    
    return N_CDF(d_1)*S - N_CDF(d_2)*K*np.exp(-r*t)